# Font recognition - base models

The code in this file will represent the basic models fitted to the data for font-recognition. This file will be submitted for grading purposes of the project.

## Load data and train-validation split

**Data is loaded, observed and treated**

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv('data/train_data.csv')
train_labels = pd.read_csv('data/train_labels.csv')
train_labels['Font'].unique()

array(['ARIAL', 'TIMES', 'SERIF', 'CAMBRIA', 'CALIBRI', 'TAHOMA'],
      dtype=object)

There are no missing labels or values

In [3]:
print(train_data.shape[0] == train_data.dropna().shape[0])
print(train_labels.shape[0] == train_labels.dropna().shape[0])

True
True


Labels are factorized

In [4]:
label_encoded, unique_labels = pd.factorize(train_labels['Font']) # unique has the index to decode labels

And a full dataframe is constructed adding the encoded values as the last column

In [5]:
labels = pd.DataFrame(label_encoded, columns=['label'])
df = pd.concat([train_data, labels], axis = 1)

**Train and validation split is conducted**

In [6]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1]
Y = df.iloc[:, -1]
x_train_df, x_valid_df, y_train_df, y_valid_df = train_test_split(X, Y, test_size=0.3, random_state = 0)
X.shape

(65000, 407)

**Finally, the test data is loaded as well**

In [7]:
test_data = pd.read_csv('data/test_data.csv')
test_data.shape

(29221, 407)

In [8]:
x_test_df = test_data

## Normalization of data

Now df has all the needed information. It will be transformed to a np.array for easier treatment within sklearn package

In [9]:
x_train_pre_norm = np.array(x_train_df)
x_valid_pre_norm = np.array(x_valid_df)
y_train = np.array(y_train_df)
y_valid = np.array(y_valid_df)

In [10]:
x_test_pre_norm = np.array(x_test_df)
X_np = np.array(X)

`mean` and `std` are obtained from full dataset

In [11]:
mean = np.sum(X_np, axis = 0) / X_np.shape[0]
std = np.std(X_np, axis = 0)

Implement normalization function from Homework 9

In [12]:
def normalize(X, mean, std):
    """Normalizes a given array X by columns 
    with the mean and std"""
    X_out = np.zeros(X.shape)
    X_out = (X - mean)/std
    return X_out 

In [13]:
x_train = normalize(x_train_pre_norm, mean, std)
x_valid = normalize(x_valid_pre_norm, mean, std)
x_test = normalize(x_test_pre_norm, mean, std)

Check normalization has been done correctly:
$$\text{mean}=0$$ $$\text{std}= 1$$

In [14]:
np.mean(x_train, axis = 0)[0:4]

array([ 0.00081244, -0.00083289, -0.00177084, -0.00353234])

In [15]:
np.std(x_valid, axis = 0)[0:4]

array([1.0004818 , 1.00024253, 1.00063935, 0.98696497])

In [16]:
np.mean(x_test, axis = 0)[0:4]

array([ 0.00514263, -0.00069739,  0.00495033, -0.00141616])

In [17]:
np.std(x_test, axis = 0)[0:4]

array([1.00905403, 0.99991203, 1.00076391, 1.00143199])

This allows us to have two different sets of features: with and without normalization. The normalization set will be used

## Function to save submission csv

A function will be created that saves predictions as a csv with the correct format

In [18]:
def predictions_as_csv(y_pred, file_name):
    path = "submissions/"
    status = 0
    if len(y_pred) == 29221:
        ids = np.arange(1,len(y_pred)+1,1)
        pred_label = unique_labels[y_pred]
        data = {'ID':ids, 'Font':pred_label} 
        submission = pd.DataFrame(data)
        submission.to_csv(path + file_name + ".csv", index = False)
        status = 1
    
    return status

## Naive Bayes

In [19]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import hamming_loss

In [20]:
model_nBayes = GaussianNB()
model_nBayes.partial_fit(x_train, y_train, np.unique(y_train))

GaussianNB()

In [21]:
y_pred_train = model_nBayes.predict(x_train)
error = hamming_loss(y_train, y_pred_train)
print('The training error is: ' + str(error) + '.')

The training error is: 0.6798021978021977.


In [22]:
y_pred_valid = model_nBayes.predict(x_valid)
error = hamming_loss(y_valid, y_pred_valid)
print('The validation error is: ' + str(error) + '.')

The validation error is: 0.6911282051282052.


In [23]:
predicted_score = 1- error
predicted_score

0.30887179487179484

**Predictions with test set are computed**

In [24]:
y_pred_test = model_nBayes.predict(x_test)

In [25]:
predictions_as_csv(y_pred_test, "naiveBayes_prediction")

1

True score was: **0.3231**

## Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import hamming_loss

In [27]:
model_logistic = LogisticRegression(multi_class='ovr', solver='liblinear')

In [28]:
model_logistic.fit(x_train, y_train)

LogisticRegression(multi_class='ovr', solver='liblinear')

In [29]:
y_pred_train = model_logistic.predict(x_train)
error = hamming_loss(y_train, y_pred_train)
print('The training error is: ' + str(error) + '.')

The training error is: 0.5043736263736264.


In [30]:
y_pred_valid = model_logistic.predict(x_valid)
error = hamming_loss(y_valid, y_pred_valid)
print('The validation error is: ' + str(error) + '.')

The validation error is: 0.5298974358974359.


In [31]:
predicted_score = 1- error
predicted_score

0.4701025641025641

**Predictions with test set are computed**

In [32]:
y_pred_test = model_logistic.predict(x_test)

In [33]:
predictions_as_csv(y_pred_test, "logistic_prediction")

1

True score was: **0.4662**